<a href="https://colab.research.google.com/github/weilainicolas/Trading/blob/main/notebooks/trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd


def find_cross_above_50_200_ma(symbol):
    try:
        today = datetime.now().date()
        # Fetch historical data
        stock_data = yf.download(symbol, period="1y")

        # Calculate moving averages
        stock_data['50MA'] = stock_data['Close'].rolling(window=50).mean()
        stock_data['200MA'] = stock_data['Close'].rolling(window=200).mean()

        # Find dates where 50MA crossed above 200MA
        cross_dates = stock_data[(stock_data['50MA'] > stock_data['200MA']) & (stock_data['50MA'].shift(1) < stock_data['200MA'].shift(1))].index

        # Check if the crossing happened within the last 5 days
        recent_cross_dates = [date for date in cross_dates if (datetime.now() - date).days <= 5]

        return symbol, recent_cross_dates
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return symbol, []

# Get list of S&P 500 symbols
sp500_symbols = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

crossed_stocks = []

# Loop through all S&P 500 symbols
for symbol in sp500_symbols['Symbol']:
    symbol, cross_dates = find_cross_above_50_200_ma(symbol)
    if cross_dates:
        crossed_stocks.append((symbol, cross_dates))

print(crossed_stocks)


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def find_stocks_rsi_below_30(symbol):
    try:
        # Fetch historical data
        stock_data = yf.download(symbol,period="1y" )

        # Calculate daily RSI
        delta = stock_data['Close'].diff(1).dropna()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)

        avg_gain = gain.rolling(window=14).mean()
        avg_loss = loss.rolling(window=14).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))

        # Check if RSI dropped below 30 within the last 3 days
        if any(rsi.iloc[-4:] < 30):
            return symbol, rsi.iloc[-1]
        else:
            return None
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Get list of NASDAQ symbols


# Find stocks with RSI below 30 within the last 3 days
stocks_rsi_below_30 = []

# Loop through all NASDAQ symbols
for symbol in sp500_symbols['Symbol']:
    result = find_stocks_rsi_below_30(symbol)
    if result:
        stocks_rsi_below_30.append(result)

# Convert results to DataFrame
df_stocks_rsi_below_30 = pd.DataFrame(stocks_rsi_below_30, columns=['Symbol', 'RSI'])

print(df_stocks_rsi_below_30)
